# Computer Vision - Autumn 2021 - Home Assignment 2

**Ondřej Schejbal & Jan Vladár**

In our assigned we deal with problem of image detection and segmentation. We apply different approaches for template matching, descriptor detection and image segmentation of selected sea plants.

For our project we decided to work with these 2 plant species:
* <a href="https://en.wikipedia.org/wiki/Zostera">Zostera</a>
* <a href="https://en.wikipedia.org/wiki/Zostera">Mytilus</a>

In [1]:
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

class1FolderName = './Mytilus_original'
class2FolderName = './Zostera_original'

In [2]:
# make the output images larger
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 300

In [3]:
def getFolderNameAndEtalonFolderName(classFolderName, print_it=False):
    folderName = classFolderName[:-9]
    etalonFolderName = folderName + 'Etalons'
    if print_it:
        print('Folder name:', folderName, '\r\nEtalon folder name:', etalonFolderName)
    return folderName, etalonFolderName

## Task 1
*Collect a set of images suitable for the tasks below of at least 2 species. Write code to preprocess the 
images of plants into a uniform size of your choice, e.g. 1024x1024 pixels.*

We have searched for relevant **Zostera** and **Mytilus** images on the web, but most of the pictures contained either some form of watermark, or some additional image distortion. Because of that we were left with only limited number of images for our task. This most likely had significant impact, especially on the deep neural network solution described in [part 4](#part_4) and that's why we decided to use data agmentation for that part of our assignment.

In the code below we are transforming the images into recommended dimensions 1024x1024 while also converting them to the *.png* format.

In [4]:
def transformImagesInDirectory(folderName):
    list_of_files = os.listdir(folderName)
    targetFolderName = folderName[:-9]
    for idx, file in enumerate(list_of_files):
        image_file_name = os.path.join(folderName, file)
        img = Image.open(image_file_name)  # .convert("L")
        img = img.resize((1024, 1024))
        if not os.path.exists(targetFolderName):
            os.mkdir(targetFolderName)
        img.save(targetFolderName + '/' + str(idx) + ".png")
        # os.remove(image_file_name)

In [5]:
def task1():
    transformImagesInDirectory(class1FolderName)
    transformImagesInDirectory(class2FolderName)
# task1()

## Task 2
*Select a set of etalons (e.g. small images containing a sample of some distinctive features) from the 
an image to be used for matching similar objects. Aim for a set that would be representative on at 
least 50% of the images of the particular species. Think how to deal with rotations.*

In this task we have manually created 9 etalons for each selected sea plant. We have selected the etalons as small as possible, but still containing representative and dominant part of given plants visual.

For Mytilus this was quite simple, but for Zostera it was not. Zostera is basically a sea grass and selecting only one grass stalk was sometimes very challenging. We have also decided to create Zostera etalons from different parts of the grass stalks - from the top, middle and bottom of the stalk.

When selecting the etalons our goal was to have a selection of small pieces from our images, which would have enough representative characteristic needed for succesfull template matching with at least 50 % accuracy for the respective plant species.

In the cells below you can see us matching the created etalons one by one to each image in our dataset. We have used the opencv2 <a href="https://docs.opencv.org/4.x/df/dfb/group__imgproc__object.html#ga586ebfb0a7fb604b35a23d85391329be">matchTemplate</a> function. This function has many matching methods and after some experiments and studying the differences between them we have decided to use the **TM_CCOEFF_NORMED** match method for this task, which represents the normalized value of correlation coefficient between the images.

<p style="display:none">https://stackoverflow.com/questions/55469431/what-does-the-tm-ccorr-and-tm-ccoeff-in-opencv-mean</p>

**Resolution**: We were able to achieve ~60 % of accuracy for Zostera and ~55 % accuracy for Mytilus.

We have also experimented with etalons. We observed that modifying the size significantly affects the accuracy. When the etalons were much bigger, the accuracy became significantly lower.

In [6]:
def match_image(img, template):
    w, h = template.shape[::-1]
    match_method = cv2.TM_CCOEFF_NORMED
    res = cv2.matchTemplate(img, template, match_method)
    _, maxval, _, maxloc = cv2.minMaxLoc(res)
    btm_right = (maxloc[0] + w, maxloc[1] + h)
    cv2.rectangle(img, maxloc, btm_right, 255, 2)
    return maxval

In [21]:
def matchEtalonToEachImage(etalons, images):
    # matching_results = []
    matching_results_by_etalon = []
    for etalon_name in os.listdir(etalons):
        etalon_path = etalons + "/" + etalon_name
        # print("etalon:", etalon_path)
        img_template = cv2.imread(etalon_path, 0) 
        matching_results = []
        for image_name in os.listdir(images):
            img_path = images + "/" + image_name
            # print("\t img", img_path)
            img = cv2.imread(img_path, 0)
            match_res = match_image(img, img_template)
            matching_results.append(match_res)
        matching_results_by_etalon.append(matching_results)

    averages = []

    for val in matching_results_by_etalon:
        averages.append(np.average(val))
    print("Precision:", np.average(averages) * 100)
    return averages

In [22]:
def task2(classFolderName):
    folderName, etalonFolderName = getFolderNameAndEtalonFolderName(classFolderName)
    precisionForEachEtalonList = matchEtalonToEachImage(etalonFolderName, folderName)
    # print("Precisions for each etalon:", precisionForEachEtalonList)

print("Mytilus")
task2(class1FolderName)

print("Zostera")
task2(class2FolderName)

Mytilus
Precision: 54.667162322081055
Zostera
Precision: 59.799787082842414


## Task 3
Use at least 3 different existing conventional feature detectors provided by 
OpenCV to find matches of the etalons in the image. NB! Take into account overlaps and subtract the 
appropriate numbers from total scores.

Evaluate on two different images (called task3a.tiff and task3b.tiff) how well the approach works and 
which feature detector performs best.

For this task we have used 3 different approaches:
1. SIFT with FlannBased matching
2. ORB
3. SIFT with BruteForce matching

## Task 4<a id='part_4'></a>
*Improve the baseline by applying deep learning.*

**TODO REMOVE THE REST**


Key words: OpenCV 4, OpenVINO, ONNX, 
Tensorflow, PyTorch. The result needs to be documented, i.e. you should present quantitative results 
in a report where you show if and how much the deep learning approach improved your baseline. 

Make sure to use appropriate criteria for the measurement!

NB! Aim at using a pretrained network as a basis and apply the concept of transfer learning to adjust 
the net for your task.

NB! The work needs to be documented, i.e. you need to include a report where you have 
quantitative results of your baseline and improvement in addition to the description of the approach 
taken.

To store images in the Git repository you should use Git LFS support.
You may use the AI-Lab environment for GPU accelerated computations.*


**Helpful hints:**

Labling tool: CVAT